# 데이터 전처리 방안 수정
- 그리드 생성 로직 변경
    - 테스트셋에 처음 등장하는 아이템에 대한 학습 : 테스트셋에 있는 아이템이 무조건 등장하도록
    - sales_train은 무언가 팔린 기록이 있는 데이터 -> 아무것도 안 팔리는 상황을 학습하는데 부족
    - shop 기준: 해당 월에 매출이 발생한 상점만 선택(폐업했거나 아직 개업 안 한 상점 제외)
    - item 기준: 해당 월에 매출이 발생한 아이템 + test 셋에 있는 모든 아이템
- 가격 변수 결측치를 ffill, bfill, 카테고리 평균 등으로 처리
- 아이템 이름 전처리 -> platform_type, meta_type -> lag_1 추가

# Module

In [1]:
import gc
import calendar
from collections import Counter
import re
from itertools import product
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

%matplotlib inline
plt.style.use("seaborn-v0_8-white")

# Data

In [2]:
data_path = "./data/"

sales_train = pd.read_csv(data_path + 'sales_train.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
shops = pd.read_csv(data_path + 'shops.csv')

test = pd.read_csv(data_path + 'test.csv')
sub = pd.read_csv(data_path + 'sample_submission.csv')

# print('Before Filter ShopID:', len(sales_train))
# unique_test_shop_id = test['shop_id'].unique()
# sales_train = sales_train[sales_train['shop_id'].isin(unique_test_shop_id)]
# print('After Filter ShopID :', len(sales_train))

In [3]:
# def downcast(df, include_floats=None, verbose=True):
#     start_memory = df.memory_usage().sum() / 1024**2
    
#     if include_floats is None:
#         include_floats = []

#     for col in df.columns:
#         dtype_name = df[col].dtype.name
        
#         if dtype_name == 'object':
#             pass
            
#         elif dtype_name == 'bool':
#             df[col] = df[col].astype('int8')
            
#         elif dtype_name.startswith('int') or (df[col]%1==0).all():
#             df[col] = pd.to_numeric(df[col], downcast='integer')
            
#         elif dtype_name.startswith('float'):
#             if col in include_floats:
#                 df[col] = pd.to_numeric(df[col], downcast='float')

#     end_memory = df.memory_usage().sum() / 1024**2
#     if verbose:
#         print(f"Memory usage reduced from {start_memory:.2f} MB to {end_memory:.2f} MB")
        
#     return df

In [4]:
def downcast(df, verbose=True):
    start_memory = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col]%1==0).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif dtype_name.startswith('float'):
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_memory = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print(f"Memory usage reduced from {start_memory:.2f} MB to {end_memory:.2f} MB")
        
    return df

## Grid

In [5]:
sales_train.loc[sales_train['shop_id']==0, 'shop_id'] = 57
sales_train.loc[sales_train['shop_id']==1, 'shop_id'] = 58
sales_train.loc[sales_train['shop_id']==10, 'shop_id'] = 11
sales_train.loc[sales_train['shop_id']==39, 'shop_id'] = 40

test.loc[test['shop_id']==0, 'shop_id'] = 57
test.loc[test['shop_id']==1, 'shop_id'] = 58
test.loc[test['shop_id']==10, 'shop_id'] = 11
test.loc[test['shop_id']==39, 'shop_id'] = 40

In [6]:
# # option1: by month
# all_shops = sales_train['shop_id'].unique()
# test_items = test['item_id'].unique()
# all_months = sales_train['date_block_num'].unique()

# matrix = []

# for i in range(34):
#     sales = sales_train[sales_train['date_block_num'] == i]
#     shops_in_month = sales['shop_id'].unique()
#     items_in_month = sales['item_id'].unique()
    
#     items_to_grid = np.unique(np.concatenate([items_in_month, test_items]))
#     matrix.append(np.array(list(product([i], shops_in_month, items_to_grid)), dtype=np.int16))
    
# idx_features = ['date_block_num', 'shop_id', 'item_id']
# train_matrix = pd.DataFrame(np.vstack(matrix), columns=idx_features)
# train_matrix.head()

In [7]:
# option2: all items x all shops
def create_grid(sales, test_df):
    months = list(range(34))
    # all_shops = sales['shop_id'].unique()
    # all_items = sales['item_id'].unique()
    grid = []
    for month in months:
        shops_in_month = sales.loc[sales['date_block_num'] == month, 'shop_id'].unique()
        items_in_month = sales.loc[sales['date_block_num'] == month, 'item_id'].unique()
        grid.append(np.array(list(product([month], shops_in_month, items_in_month)), dtype='int32'))

    # 월별로 생성된 그리드를 결합
    grid_df = pd.DataFrame(np.vstack(grid), columns=['date_block_num', 'shop_id', 'item_id'])
    
    # 테스트셋에만 존재하는 (상점, 아이템) 조합을 그리드에 추가
    test_shops = test_df['shop_id'].unique()
    test_items = test_df['item_id'].unique()
    test_grid_addition = []
    for month in months:
        test_grid_addition.append(np.array(list(product([month], test_shops, test_items)), dtype='int32'))
        
    test_grid_df = pd.DataFrame(np.vstack(test_grid_addition), columns=['date_block_num', 'shop_id', 'item_id'])
    full_grid = pd.concat([grid_df, test_grid_df], ignore_index=True, sort=False).drop_duplicates()

    return full_grid


train_matrix = create_grid(sales_train, test)
idx_features = ['date_block_num', 'shop_id', 'item_id']
print('='*10, 'Matrix Table', '='*10)
print(train_matrix.head())
print('='*10, 'Matrix Table Info', '='*10)
print(train_matrix.info())

========== Matrix Table ==========
   date_block_num  shop_id  item_id
0               0       59    22154
1               0       59     2552
2               0       59     2554
3               0       59     2555
4               0       59     2564
========== Matrix Table Info ==========
<class 'pandas.core.frame.DataFrame'>
Index: 15343898 entries, 0 to 18124908
Data columns (total 3 columns):
 #   Column          Dtype
---  ------          -----
 0   date_block_num  int32
 1   shop_id         int32
 2   item_id         int32
dtypes: int32(3)
memory usage: 292.7 MB
None


In [8]:
# Origin
# train = []

# for i in sales_train['date_block_num'].unique():
#     all_shop = sales_train.loc[sales_train['date_block_num'] == i, 'shop_id'].unique()
#     all_item = sales_train.loc[sales_train['date_block_num'] == i, 'item_id'].unique()
#     train.append(np.array(list(product([i], all_shop, all_item))))

# # idx features        
# idx_features = ['date_block_num', 'shop_id', 'item_id']
# train = pd.DataFrame(np.vstack(train), columns=idx_features)

# test['date_block_num'] = 34
# train.head()

# Feature Engineering

## Outliers
- `-1` 같은 음수값은 반품을 의미할 수 있음. 제거하지 않고 `item_cnt_month`로 집계

In [9]:
print("Before : ",len(sales_train))
sales_train = sales_train[sales_train['item_cnt_day'] < 1000]
print("After item_cnt_day : ",len(sales_train))
sales_train = sales_train[sales_train['item_price'] < 50000]
print("After item_price : ",len(sales_train))

Before :  2935849
After item_cnt_day :  2935847


After item_price :  2935844


## Define function

In [10]:
def add_mean_features(df: pd.DataFrame, groupby_features: list, mean_feature_list: list = None):
    col_name = [col for col in groupby_features if col != 'date_block_num']
    base_name = '_'.join([col.replace('_id', '') for col in col_name])
    feature_name = f'{base_name}_avg_date_sales'
    
    agg_rules = {
        feature_name: ('item_cnt_month', 'mean')
    }
    group = df.groupby(groupby_features).agg(**agg_rules).reset_index()
    df = df.merge(group, on=groupby_features, how='left')
    del group
    
    if mean_feature_list is not None:
        mean_feature_list.append(feature_name)
    return df


def add_lag_features(df: pd.DataFrame, key_features: list, lag_feature_cols: list, lag_period: list):
    df_result = df.copy()

    for i in lag_period:
        df_lag = df[key_features + lag_feature_cols].copy()
        df_lag['date_block_num'] += i
        lag_col_names = {col: f'{col}_lag_{i}' for col in lag_feature_cols}
        df_lag = df_lag.rename(columns=lag_col_names)
        df_result = pd.merge(df_result, df_lag, on=key_features, how='left')

    all_lag_cols = [f'{col}_lag_{i}' for col in lag_feature_cols for i in lag_period]
    for col in all_lag_cols:
        if 'cnt' in col or 'sales' in col:
            df_result[col] = df_result[col].fillna(0)
    return df_result

## shops

In [ ]:
# 데이터 수정
shops['city'] = shops['shop_name'].str.split(' ').str[0]
shops.loc[shops['city'] == '!Якутск', 'city'] = 'Якутск'

# 특수 상점(온라인/이동식)을 별도 도시 'Special'로 분류
shops.loc[shops['city'].isin(['Выездная', 'Интернет-магазин']), 'city'] = 'Special'

# 약어 및 오표기 수정
shops.loc[shops['city'] == 'СПб', 'city'] = 'Санкт-Петербург'
shops.loc[shops['city'] == 'Н.Новгород', 'city'] = 'НижнийНовгород'
shops.loc[shops['city'] == 'РостовНаДону', 'city'] = 'Ростов-на-Дону'

# 모스크바 위성 도시 통합
moscow_satellite_cities = ['Жуковский', 'Мытищи', 'Химки', 'Чехов', 'Балашиха', 'Сергиев']
shops.loc[shops['city'].isin(moscow_satellite_cities), 'city'] = 'МоскваОбласть'

# 쇼핑몰/센터를 나타내는 러시아어 약어로 상점 구분
# mall_keywords = ['ТЦ', 'ТРК', 'ТРЦ', 'MALL', 'Молл']
# is_mall = shops['shop_name'].apply(lambda x: any(keyword in x for keyword in mall_keywords))

# shops['shop_type'] = np.where(is_mall, 'Mall', 'Standalone')

for col in ['city']: # , 'shop_type'
    encoder = LabelEncoder()
    shops[col] = encoder.fit_transform(shops[col])

shops = shops.drop(columns=["shop_name"])

## items

In [12]:
first_sale_mon = sales_train.groupby('item_id').agg({'date_block_num': 'min'})['date_block_num']
items['first_sale_month'] = items['item_id'].map(first_sale_mon).fillna(34)

##### 월별 아이템 평균가격

In [13]:
date_item_avg_price = sales_train.groupby(['date_block_num', 'item_id']).agg(date_item_avg_price=('item_price', 'mean')).reset_index()

### meta, platform

In [14]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9а-яА-Я\s]', ' ', text) 
    text = re.sub(r'\s+', ' ', text).strip()
    return text

items['cleaned_item_name'] = items['item_name'].apply(clean_text)
all_words = ' '.join(items['cleaned_item_name'].values).split()
Counter(all_words).most_common(50)

[('версия', 3599),
 ('pc', 2683),
 ('bd', 2322),
 ('цифровая', 2003),
 ('регион', 1849),
 ('2', 1821),
 ('русская', 1452),
 ('jewel', 1370),
 ('и', 1251),
 ('1', 1244),
 ('the', 1185),
 ('3', 1133),
 ('1с', 1120),
 ('cd', 1019),
 ('of', 955),
 ('в', 949),
 ('mp3', 948),
 ('dvd', 899),
 ('фирм', 757),
 ('xbox', 754),
 ('ps3', 735),
 ('edition', 681),
 ('фигурка', 667),
 ('для', 638),
 ('s', 615),
 ('коллекция', 576),
 ('360', 571),
 ('3d', 566),
 ('digipack', 553),
 ('4', 551),
 ('на', 535),
 ('lp', 524),
 ('с', 504),
 ('арт', 465),
 ('2cd', 452),
 ('a', 437),
 ('русские', 420),
 ('игра', 418),
 ('сб', 406),
 ('субтитры', 402),
 ('7', 367),
 ('v', 367),
 ('5', 356),
 ('футболка', 356),
 ('игрушка', 352),
 ('английская', 345),
 ('box', 336),
 ('набор', 334),
 ('8', 332),
 ('издание', 308)]

- 대상이 되는 1C COMPANY는 게임 소프트웨어를 취급하는 러시아 기업
- pc(2위), цифровая(디지털, 4위) -> 디지털 다운로드는 cd/dvd 제품과 판매형태가 전혀 다름. 온라인 위주로 판매될 것
- xbox(19위), ps3는 콘솔게임기고 360(46위)도 xbox360 콘솔을 지칭하는 것으로 추정됨
- jewel(8위, 저가판), dvd(18위), cd(14위), bd(블루레이 디스크, 3위), box(47위), edition(41위) 등은 제품형태를 뜻하는 걸로 보임
    - 상대적으로 저렴한 저가판의 판매가 많고, 고가의 에디션은 판매량이 상대적으로 적을 것으로 추정
- фигурка (피규어, 23위), футболка (티셔츠, 44위), игрушка (장난감, 45위) -> 게임 및 콘솔과 판매 패턴이 다를 것

In [15]:
def get_platform(name):
    if 'pc' in name or 'пк' in name: return 'PC'
    if 'ps3' in name: return 'PS3'
    if 'ps4' in name: return 'PS4'
    if 'xbox' in name or 'x360' in name: return 'Xbox' # 360도 Xbox로 통합
    if 'psp' in name: return 'PSP'
    if 'vita' in name or 'psv' in name: return 'PSVita'
    if 'wii' in name: return 'Wii'
    if 'mac' in name: return 'Mac'
    if 'android' in name: return 'Android'
    return 'Etc'

def get_meta_type(name):
    # 디지털/버전 정보
    if 'цифровая' in name or 'digital' in name: return 'Digital'
    if 'версия' in name: return 'Version' # Version 보통 일반판   
    # 실물 미디어
    if 'bd' in name or 'blu-ray' in name: return 'BluRay'
    if 'dvd' in name: return 'DVD'
    if 'cd' in name: return 'CD'
    if 'lp' in name: return 'Vinyl'
    # 패키지 형태
    if 'jewel' in name: return 'Jewel' # 저가판
    if 'region' in name or 'регион' in name: return 'Region' # 현지화/지역한정
    if 'edition' in name or 'издание' in name: return 'Edition' # 특별판 등
    if 'box' in name: return 'Box'
    # 굿즈
    if 'фигурка' in name: return 'Figure'
    if 'футболка' in name: return 'TShirt'
    if 'игрушка' in name: return 'Toy'
    if 'арт' in name: return 'Art'
    return 'Normal'


items['platform_type'] = items['cleaned_item_name'].apply(get_platform)
items['meta_type'] = items['cleaned_item_name'].apply(get_meta_type)
for col in ['platform_type', 'meta_type']:
    items[col] = LabelEncoder().fit_transform(items[col])
items = items.drop(columns=['item_name', 'cleaned_item_name'])

## item_categories

In [16]:
def clean_category_type(df):
    df['type'] = df['item_category_name'].apply(lambda x: x.split('-')[0].strip())
    df['type'] = df['type'].apply(lambda x: 'Игры' if 'Игры' in x else x) # 모든 게임(PC, Android, Mac)을 'Игры'로 통합
    df['type'] = df['type'].apply(lambda x: 'Карты оплаты' if 'Карты' in x else x) # 카드류 통합
    df['type'] = df['type'].apply(lambda x: 'Чистые носители' if 'Чистые' in x else x) # 미디어류 통합
    print(df['type'].value_counts())
    common_types = df['type'].value_counts()
    etc_types = common_types[common_types < 4].index.tolist()
    df['type'] = df['type'].apply(lambda x: 'etc' if x in etc_types else x)
    return df

item_categories = clean_category_type(item_categories)
item_categories['type'] = LabelEncoder().fit_transform(item_categories['type'])
item_categories = item_categories.drop('item_category_name', axis=1)

type
Игры                15
Книги               13
Подарки             12
Игровые консоли      8
Аксессуары           7
Музыка               6
Программы            6
Кино                 5
Карты оплаты         4
Служебные            2
Чистые носители      2
PC                   1
Билеты (Цифра)       1
Доставка товара      1
Элементы питания     1
Name: count, dtype: int64


## revenue

In [17]:
sales_train['revenue'] = sales_train['item_cnt_day'] * sales_train['item_price']

## set up matrix

In [18]:
data_files = [sales_train, shops, items, item_categories, train_matrix]
for file in data_files:
    file = downcast(file)

Memory usage reduced from 179.19 MB to 106.39 MB
Memory usage reduced from 0.00 MB to 0.00 MB
Memory usage reduced from 0.85 MB to 0.13 MB
Memory usage reduced from 0.00 MB to 0.00 MB
Memory usage reduced from 292.66 MB to 175.60 MB


In [19]:
# new grid
group = sales_train.groupby(idx_features).agg(
    item_cnt_month=('item_cnt_day', 'sum'),
    transaction_cnt=('item_cnt_day', 'count'),
    date_revenue=('revenue', 'sum')
).reset_index()

train_matrix = train_matrix.merge(group, on=idx_features, how='left')
train_matrix['item_cnt_month'] = train_matrix['item_cnt_month'].fillna(0) # 판매량 결측치
train_matrix['item_cnt_month'] = train_matrix['item_cnt_month'].clip(0, 20) # 클리핑 미리

# 메모리 최적화
train_matrix['date_block_num'] = train_matrix['date_block_num'].astype(np.int8)
train_matrix['shop_id'] = train_matrix['shop_id'].astype(np.int8)
train_matrix['item_id'] = train_matrix['item_id'].astype(np.int16)
train_matrix['item_cnt_month'] = train_matrix['item_cnt_month'].astype(np.float16)

test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)
test['item_cnt_month'] = 0.0
test['item_cnt_month'] = test['item_cnt_month'].astype(np.float16)

all_data = pd.concat([train_matrix, test.drop(columns=['ID'])], ignore_index=True, sort=False)
all_data.fillna(0, inplace=True)
all_data = all_data.merge(shops, on='shop_id', how='left')
all_data = all_data.merge(items, on='item_id', how='left')
all_data = all_data.merge(date_item_avg_price, on=['date_block_num', 'item_id'], how='left')
all_data = all_data.merge(item_categories, on='item_category_id', how='left')

# 최종 데이터 확인
print("--- 최종 데이터 정보 ---")
all_data.info()
print("\n--- 최종 데이터 샘플 ---")
print(all_data.tail())

--- 최종 데이터 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15558098 entries, 0 to 15558097
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   date_block_num       int8   
 1   shop_id              int8   
 2   item_id              int16  
 3   item_cnt_month       float16
 4   transaction_cnt      float64
 5   date_revenue         float64
 6   city                 int8   
 7   item_category_id     int8   
 8   first_sale_month     int8   
 9   platform_type        int8   
 10  meta_type            int8   
 11  date_item_avg_price  float64
 12  type                 int8   
dtypes: float16(1), float64(3), int16(1), int8(8)
memory usage: 534.1 MB

--- 최종 데이터 샘플 ---
          date_block_num  shop_id  item_id  item_cnt_month  transaction_cnt  \
15558093              34       45    18454             0.0              0.0   
15558094              34       45    16188             0.0              0.0   
15558095              34    

In [20]:
# # old grid
# group = sales_train.groupby(idx_features).agg(
#     item_cnt_month = ('item_cnt_day', 'sum'),
#     transaction_cnt = ('item_cnt_day', 'count'),
#     date_revenue = ('revenue', 'sum')
# ).reset_index()

# train = train.merge(group, on=idx_features, how='left')
# all_data = pd.concat([train, test.drop('ID', axis=1)], ignore_index=True, keys=idx_features)
# all_data = all_data.merge(shops, on='shop_id', how='left')
# all_data = all_data.merge(items, on='item_id', how='left')
# all_data = all_data.merge(date_item_avg_price, on=['date_block_num', 'item_id'], how='left')
# all_data = all_data.merge(item_categories, on='item_category_id', how='left')
# all_data.head()

## fill nan price feature

In [21]:
temp = all_data.isna().sum()
temp[temp > 0]

date_item_avg_price    4572549
dtype: int64

In [22]:
def fill_price_nan(df):
    # 처리할 모든 가격 관련 컬럼 식별
    price_cols = ['date_item_avg_price'] + [col for col in df.columns if 'price_lag_' in col]

    df = df.sort_values(by=['shop_id', 'item_id', 'date_block_num'])
    df[price_cols] = df.groupby(['shop_id', 'item_id'])[price_cols].ffill()
    # df[price_cols] = df.groupby(['shop_id', 'item_id'])[price_cols].bfill()

    # 데이터 누수 최소화를 위해 기간 단축
    train_data = df[df['date_block_num'] < 33]
    # 아이템별 전체 평균
    item_overall_avg_price_map = train_data.groupby('item_id')['date_item_avg_price'].mean()
    # 카테고리별 전체 평균
    cat_overall_avg_price_map = train_data.groupby('item_category_id')['date_item_avg_price'].mean()
    # 전역 평균
    global_avg_price = train_data['date_item_avg_price'].mean()
    
    for col in price_cols:
        df[col].fillna(df['item_id'].map(item_overall_avg_price_map), inplace=True)
        df[col].fillna(df['item_category_id'].map(cat_overall_avg_price_map), inplace=True)
        df[col].fillna(global_avg_price, inplace=True)

    del train_data, item_overall_avg_price_map, cat_overall_avg_price_map
    gc.collect()

    return df


all_data = fill_price_nan(all_data)

In [23]:
lag_1_list = []
lag_3_list = []
long_lag_list = []
features_to_drop = []

lag_3_list.extend(['transaction_cnt', 'date_item_avg_price'])
long_lag_list.extend(['item_cnt_month'])

all_data = downcast(all_data)
del shops, items, item_categories, group

Memory usage reduced from 652.84 MB to 534.14 MB


## month, days

In [24]:
all_data['month'] = all_data['date_block_num'] % 12
# days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
# all_data['days'] = all_data['month'].map(days).astype(np.int8)
all_data.groupby('month')['item_cnt_month'].mean().reset_index()

,month,item_cnt_month
0,0,0.228580
1,1,0.212143
2,2,0.221940
3,3,0.182514
4,4,0.185639
5,5,0.194358
6,6,0.189267
7,7,0.213281
8,8,0.202719
9,9,0.206917


- 12월의 평균 판매량이 특히 높게 나타남. 1월도 비교적 높음
- 3월과 9월의 평균 판매량이 상대적으로 높음(신학기 영향 예상)
- 테스트 데이터에 해당하는 11월은 평균 판매량이 1년 중 가장 낮음

In [25]:
# # 0: Low (11월 포함), 1: Normal, 2: High, 3: Peak
# season_dict = {
#     11: 3, # 연말
#     0: 2, 2: 2, 8: 2, # 0: 연초 / 2, 8: 신학기 
#     7: 2, 1:2, # 7: 저조한 6대비 판매량 회복. 신학기 준비
#     3: 1, 4: 1, 5: 1, 6:1, 9: 1, # 평범
#     10: 0  # 6: 비수기, 휴가철 / 10: pre holiday
# }

# all_data['season_type'] = all_data['month'].map(season_dict).astype('int8')

In [26]:
# def count_weekends(date_block_num):
#     year = 2013 + date_block_num // 12
#     month = 1 + date_block_num % 12
    
#     _, last_day = calendar.monthrange(year, month)
#     dates = pd.date_range(start=f'{year}-{month}-01', end=f'{year}-{month}-{last_day}')
#     weekend_count = dates.weekday.isin([5, 6]).sum()
    
#     return weekend_count

# weekend_map = {i: count_weekends(i) for i in range(35)}
# all_data['num_weekends'] = all_data['date_block_num'].map(weekend_map).astype('int8')
# all_data[['date_block_num', 'month', 'num_weekends']].head()

In [27]:
# all_data['num_weekends'].value_counts()

## mean features

In [28]:
all_data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'transaction_cnt', 'date_revenue', 'city', 'item_category_id',
       'first_sale_month', 'platform_type', 'meta_type', 'date_item_avg_price',
       'type', 'month'],
      dtype='object')

In [29]:
mean_feature_groups = [
    ['date_block_num', 'shop_id'],
    ['date_block_num', 'shop_id', 'item_category_id'],
    ['date_block_num', 'shop_id', 'type'],
    ['date_block_num', 'item_id'],
    ['date_block_num', 'item_category_id'],
    ['date_block_num', 'city', 'item_id'],
    ['date_block_num', 'city', 'type'],
    ['date_block_num', 'shop_id', 'platform_type'],
    ['date_block_num', 'shop_id', 'meta_type'],
]
for group in mean_feature_groups:
    all_data = add_mean_features(all_data, group)

In [30]:
mean_feature_list = [col for col in all_data.columns if '_avg_date_sales' in col and 'item_avg_date_sales' not in col]
long_lag_list.append('item_avg_date_sales')
mean_feature_list.append('city_item_avg_date_sales')
print('Mean Features:', len(mean_feature_list), mean_feature_list)

Mean Features: 8 ['shop_avg_date_sales', 'shop_item_category_avg_date_sales', 'shop_type_avg_date_sales', 'item_category_avg_date_sales', 'city_type_avg_date_sales', 'shop_platform_type_avg_date_sales', 'shop_meta_type_avg_date_sales', 'city_item_avg_date_sales']


## shop revenue share

In [31]:
total_rev = all_data.groupby('date_block_num')['date_revenue'].sum().reset_index()
total_rev.columns = ['date_block_num', 'total_revenue']

shop_rev = all_data.groupby(['date_block_num', 'shop_id'])['date_revenue'].sum().reset_index()
shop_rev.columns = ['date_block_num', 'shop_id', 'shop_revenue_month']

shop_rev = pd.merge(shop_rev, total_rev, on='date_block_num', how='left')
shop_rev['shop_revenue_share'] = shop_rev['shop_revenue_month'] / shop_rev['total_revenue']
shop_rev['shop_revenue_share'] = shop_rev['shop_revenue_share'].fillna(0)

all_data = pd.merge(all_data, shop_rev[['date_block_num', 'shop_id', 'shop_revenue_share']], on=['date_block_num', 'shop_id'], how='left')

del total_rev, shop_rev
gc.collect()
lag_1_list.append('shop_revenue_share')

## Lag Features

In [32]:
lag_1_list.extend(mean_feature_list)
print('Long Lag Features:', len(long_lag_list), long_lag_list)
print('Lag 1 Features:', len(lag_1_list), lag_1_list)
print('Lag 3 Features:', len(lag_3_list), lag_3_list)

Long Lag Features: 2 ['item_cnt_month', 'item_avg_date_sales']
Lag 1 Features: 9 ['shop_revenue_share', 'shop_avg_date_sales', 'shop_item_category_avg_date_sales', 'shop_type_avg_date_sales', 'item_category_avg_date_sales', 'city_type_avg_date_sales', 'shop_platform_type_avg_date_sales', 'shop_meta_type_avg_date_sales', 'city_item_avg_date_sales']
Lag 3 Features: 2 ['transaction_cnt', 'date_item_avg_price']


In [33]:
long_lag_list

['item_cnt_month', 'item_avg_date_sales']

In [34]:
%%time
all_data = downcast(all_data)

Memory usage reduced from 1617.27 MB to 1023.78 MB
CPU times: user 2.49 s, sys: 816 ms, total: 3.31 s
Wall time: 3.3 s


In [35]:
%%time
all_data = all_data.sort_values(by=['date_block_num', 'shop_id', 'item_id']).reset_index(drop=True)

all_data = add_lag_features(all_data, 
                            key_features=idx_features,
                            lag_feature_cols=lag_1_list,
                            lag_period=[1])

all_data = downcast(all_data)

Memory usage reduced from 1557.92 MB to 1557.92 MB
CPU times: user 16.5 s, sys: 4.11 s, total: 20.6 s
Wall time: 20.6 s


In [36]:
%%time
all_data = all_data.sort_values(by=['date_block_num', 'shop_id', 'item_id']).reset_index(drop=True)
all_data = add_lag_features(all_data, 
                            key_features=idx_features,
                            lag_feature_cols=lag_3_list,
                            lag_period=[1, 2, 3])

all_data = downcast(all_data)

Memory usage reduced from 2270.12 MB to 1958.53 MB
CPU times: user 20.2 s, sys: 9.93 s, total: 30.1 s
Wall time: 30.2 s


In [37]:
%%time
all_data = all_data.sort_values(by=['date_block_num', 'shop_id', 'item_id']).reset_index(drop=True)
all_data = add_lag_features(all_data, 
                            key_features=idx_features,
                            lag_feature_cols=long_lag_list,
                            lag_period=[1,2,3,6,12])

all_data['shop_revenue_share_lag_1'].fillna(0, inplace=True)

CPU times: user 27.4 s, sys: 21.6 s, total: 49 s
Wall time: 49.6 s


In [38]:
features_to_drop.extend(lag_1_list)
features_to_drop.extend(lag_3_list)
features_to_drop.extend(long_lag_list)
features_to_drop.remove('item_cnt_month')
features_to_drop.remove('date_item_avg_price')

all_data = all_data.drop(columns=features_to_drop)
gc.collect()

all_data = all_data.copy()
features_to_drop = []

## price trend

In [39]:
temp = all_data.isna().sum()
temp[temp > 0]

date_item_avg_price_lag_1    2287873
date_item_avg_price_lag_2    2954060
date_item_avg_price_lag_3    3612060
dtype: int64

In [40]:
price_lag_cols = [col for col in all_data.columns if 'date_item_avg_price_lag' in col]
all_data = fill_price_nan(all_data)

temp = all_data.isna().sum()
temp[temp > 0]

Series([], dtype: int64)

In [41]:
# price trend
all_data = all_data.sort_values(by=['item_id', 'date_block_num'])
df_temp = all_data[['item_id', 'date_item_avg_price']].copy()


grp = df_temp.groupby('item_id')['date_item_avg_price']
cumsum = grp.cumsum()
cumcount = df_temp['date_item_avg_price'].notnull().astype(int).groupby(df_temp['item_id']).cumsum()
df_temp['date_item_price_expanding_mean'] = cumsum / cumcount
df_temp['date_item_price_expanding_mean'] = df_temp['date_item_price_expanding_mean'].shift(1)

# 아이템 변경 지점 처리
mask = df_temp['item_id'] != df_temp['item_id'].shift(1)
df_temp.loc[mask, 'date_item_price_expanding_mean'] = 0
all_data['item_avg_price_expanding'] = df_temp['date_item_price_expanding_mean']


# 시나리오: expanding = 0 -> 미출시 혹은 신상품 -> nan -> 이전 시간 대비 가격변화 없음 -> 0으로 대체 
temp_lag_1 = all_data['date_item_avg_price_lag_1'].fillna(all_data['item_avg_price_expanding'])
all_data['delta_price_lag'] = (temp_lag_1 - all_data['item_avg_price_expanding']) / all_data['item_avg_price_expanding']
all_data['delta_price_lag'] = all_data['delta_price_lag'].replace([np.inf, -np.inf], np.nan).fillna(0)

del df_temp, grp, cumsum, cumcount, mask, temp_lag_1
gc.collect()

all_data[['item_id', 'date_block_num', 'delta_price_lag']].head()

,item_id,date_block_num,delta_price_lag
10318199,0,20,0.0
10327249,0,20,0.0
10336299,0,20,0.0
10345349,0,20,0.0
10354399,0,20,0.0


In [42]:
all_data['delta_price_lag'].isna().sum()

0

In [43]:
len(all_data[all_data['delta_price_lag']==0])

5119012

## shop revenue trend

In [44]:
all_data = all_data.sort_values(by=['shop_id', 'item_id', 'date_block_num'])

all_shop = all_data['shop_id'].unique()
all_months = all_data['date_block_num'].unique()
full_shop_df = pd.DataFrame(list(product(all_shop, all_months)), columns=['shop_id', 'date_block_num'])

shop_monthly_raw = all_data.groupby(['shop_id', 'date_block_num'])['date_revenue'].sum().reset_index().rename(columns={'date_revenue':'shop_revenue'})
shop_monthly = full_shop_df.merge(shop_monthly_raw, on=['shop_id', 'date_block_num'], how='left')
shop_monthly['shop_revenue'] = shop_monthly['shop_revenue'].fillna(0)

shop_monthly = shop_monthly.sort_values(by=['shop_id', 'date_block_num'])
grp = shop_monthly.groupby('shop_id')['shop_revenue']
cumsum = grp.cumsum()
cumcount = grp.cumcount() + 1
shop_monthly['shop_rev_expanding_mean'] = cumsum / cumcount

shop_monthly['shop_rev_expanding_mean'] = shop_monthly['shop_rev_expanding_mean'].shift(1)
mask = shop_monthly['shop_id'] != shop_monthly['shop_id'].shift(1)
shop_monthly.loc[mask, 'shop_rev_expanding_mean'] = 0
shop_monthly['shop_rev_expanding_mean'] = shop_monthly['shop_rev_expanding_mean'].fillna(0)

shop_monthly['shop_revenue_lag_1'] = grp.shift(1).fillna(0)
shop_monthly['delta_shop_revenue_lag'] = (shop_monthly['shop_revenue_lag_1'] - shop_monthly['shop_rev_expanding_mean']) / shop_monthly['shop_rev_expanding_mean']
shop_monthly['delta_shop_revenue_lag'] = shop_monthly['delta_shop_revenue_lag'].replace([np.inf, -np.inf], np.nan).fillna(0)

all_data = pd.merge(all_data, shop_monthly[['shop_id', 'date_block_num', 'delta_shop_revenue_lag']], 
                    on=['shop_id', 'date_block_num'], how='left')

del all_shop, all_months, full_shop_df, shop_monthly_raw, shop_monthly, grp, cumsum, cumcount, mask
features_to_drop.append('date_revenue')
gc.collect()
all_data[['shop_id', 'date_block_num', 'delta_shop_revenue_lag']].head()

,shop_id,date_block_num,delta_shop_revenue_lag
0,2,20,0.000553
1,2,15,0.588033
2,2,18,0.054119
3,2,19,-0.076263
4,2,20,0.000553


## Item Age

In [45]:
# item_age
all_data['item_age'] = all_data['date_block_num'] - all_data['first_sale_month']

all_data.loc[all_data['item_age'] <= 0, 'item_age'] = 0 # 출시전 기간을 모두 0으로 처리 -> 출시(혹은 판매)까지 남은 기간을 모델이 알 수 없도록

In [46]:
# item_shop_age
shop_first_sale = sales_train.groupby(['shop_id', 'item_id'])['date_block_num'].min()
all_data['item_shop_first_sale'] = all_data.set_index(['shop_id', 'item_id']).index.map(shop_first_sale)
all_data['item_shop_first_sale'] = all_data['item_shop_first_sale'].fillna(34)
all_data['item_shop_age'] = all_data['date_block_num'] - all_data['item_shop_first_sale']
all_data.loc[all_data['item_shop_age'] <= 0, 'item_shop_age'] = 0

features_to_drop.extend(['first_sale_month', 'item_shop_first_sale'])
all_data[['date_block_num', 'item_age', 'item_shop_age']].head()

,date_block_num,item_age,item_shop_age
0,20,0,0.0
1,15,0,0.0
2,18,3,0.0
3,19,4,0.0
4,20,5,0.0


## Last Sale

In [47]:
temp_df = all_data[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month']].copy()
temp_df = temp_df.sort_values(by=['shop_id', 'item_id', 'date_block_num']).reset_index(drop=True)

temp_df['item_shop_last_sale'] = np.nan
temp_df.loc[temp_df['item_cnt_month'] > 0, 'item_shop_last_sale'] = temp_df['date_block_num']
last_sale_record = temp_df.groupby(['item_id', 'shop_id'])['item_shop_last_sale'].shift(1).ffill()
temp_df['item_shop_last_sale'] = (temp_df['date_block_num'] - last_sale_record)
temp_df['item_shop_last_sale'] = temp_df['item_shop_last_sale'].fillna(-1)

all_data = pd.merge(
    all_data, 
    temp_df[['date_block_num', 'shop_id', 'item_id', 'item_shop_last_sale']],
    on=['date_block_num', 'shop_id', 'item_id'],
    how='left'
)

del temp_df, last_sale_record

In [48]:
# 출시 후 시간이 지날수록 판매량 가중치를 줄이는 피처
temp_sales_proxy = np.where(
    all_data['item_cnt_month_lag_1'] > 0, 
    all_data['item_cnt_month_lag_1'], 
    all_data['item_category_avg_date_sales_lag_1']
)


all_data['decay_rate_feature'] = temp_sales_proxy / (np.e + all_data['item_age'])
print(all_data[['item_id', 'item_cnt_month', 'item_cnt_month_lag_1', 'decay_rate_feature', 'item_age', 'item_shop_age']].head(10))

   item_id  item_cnt_month  item_cnt_month_lag_1  decay_rate_feature  \
0        0               0                   0.0            0.000000   
1        1               0                   0.0            0.000000   
2        1               0                   0.0            0.000000   
3        1               0                   0.0            0.005426   
4        1               0                   0.0            0.005217   
5        1               0                   0.0            0.004894   
6        2               0                   0.0            0.000000   
7        2               0                   0.0            0.000000   
8        3               0                   0.0            0.000000   
9        3               0                   0.0            0.041230   

   item_age  item_shop_age  
0         0            0.0  
1         0            0.0  
2         3            0.0  
3         4            0.0  
4         5            0.0  
5         6            0.0  
6   

## rolling mean/std & lag_1 and lag_12

In [49]:
all_data = all_data.drop(columns=features_to_drop)
features_to_drop = []
all_data = downcast(all_data)

Memory usage reduced from 2833.94 MB to 1928.86 MB


In [50]:
# 최근 3 평균 판매량 & 표준편차 item_cnt_month_lag
all_data['rolling_3m_cnt_mean'] = all_data[['item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3']].mean(axis=1)
all_data['rolling_3m_cnt_std'] = all_data[['item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3']].std(axis=1)

# 최근 3개월 평균 item_avg_date_sales
all_data['rolling_3m_item_mean'] = all_data[['item_avg_date_sales_lag_1', 'item_avg_date_sales_lag_2', 'item_avg_date_sales_lag_3']].mean(axis=1)
all_data['rolling_3m_item_std'] = all_data[['item_avg_date_sales_lag_1', 'item_avg_date_sales_lag_2', 'item_avg_date_sales_lag_3']].std(axis=1)
all_data = downcast(all_data)

Memory usage reduced from 2284.95 MB to 2166.25 MB


In [51]:
# 최근 3개월 월별 아이템 가격 평균 & 표준편차
all_data['rolling_3m_price_mean'] = all_data[['date_item_avg_price_lag_1', 'date_item_avg_price_lag_2', 'date_item_avg_price_lag_3']].mean(axis=1)
all_data['rolling_3m_price_std'] = all_data[['date_item_avg_price_lag_1', 'date_item_avg_price_lag_2', 'date_item_avg_price_lag_3']].std(axis=1)
features_to_drop.extend(['date_item_avg_price'])
all_data = downcast(all_data)

Memory usage reduced from 2403.65 MB to 2344.30 MB


## items per transaction

In [52]:
sales_col = ['item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3']
transaction_col = ['transaction_cnt_lag_1', 'transaction_cnt_lag_2', 'transaction_cnt_lag_3']
new_cols_dict = {}

for i in range(3):
    col_name = f'items_per_transaction_lag_{i+1}'
    new_series = all_data[sales_col[i]] / all_data[transaction_col[i]]
    new_cols_dict[col_name] = new_series.replace([np.inf, -np.inf], np.nan).fillna(0)

all_data = all_data.assign(**new_cols_dict)
all_data = all_data.copy()

## removing columns

In [53]:
features_to_drop

['date_item_avg_price']

In [54]:
print(sorted([col for col in all_data.columns if '_lag_2' in col or '_lag_3' in col]))

['date_item_avg_price_lag_2', 'date_item_avg_price_lag_3', 'item_avg_date_sales_lag_2', 'item_avg_date_sales_lag_3', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3', 'items_per_transaction_lag_2', 'items_per_transaction_lag_3', 'transaction_cnt_lag_2', 'transaction_cnt_lag_3']


In [55]:
all_data = all_data.drop(columns=features_to_drop)
features_to_drop = []

target_cols = [
    col for col in all_data.columns 
    if re.search(r'_lag_[2-3]', col)
]
# 오래된 시차 변수들 제거
features_to_drop.extend(target_cols)
features_to_drop.remove('item_avg_date_sales_lag_2')
features_to_drop.remove('item_cnt_month_lag_2')
features_to_drop.remove('item_cnt_month_lag_3')
features_to_drop.remove('transaction_cnt_lag_2')
all_data = all_data.drop(columns=features_to_drop)

print("Shape after dropping lag features:", all_data.shape)

Shape after dropping lag features: (15558098, 45)


## expanding means

In [56]:
def add_expanding_mean_features(df, group_cols, target_col='item_cnt_month'):
    df = df.sort_values(by=['date_block_num', 'shop_id', 'item_id'])
    
    col_name = [col for col in group_cols if col != 'date_block_num']
    base_name = '_'.join([col.replace('_id', '') for col in col_name])
    feature_name = f'{base_name}_expanding_mean'
    
    grouped = df.groupby(group_cols)[target_col]
    df[feature_name] = grouped.cumsum().shift(1) / grouped.cumcount().shift(1)
    df[feature_name] = df[feature_name].fillna(0)
    
    return df

In [57]:
all_data = downcast(all_data)
gc.collect()

expanding_feature_groups = [
    ['shop_id'],
    ['item_category_id'],
    ['item_id'],
    ['shop_id', 'item_category_id'],
]
for group in expanding_feature_groups:
    all_data = add_expanding_mean_features(all_data, group)

Memory usage reduced from 2032.72 MB to 1973.37 MB


## `date_block_num` >= 12

In [58]:
temp_all_data = all_data.copy()
all_data = all_data[all_data['date_block_num']>=12]
print(f"Shape after filtering: {all_data.shape}")

Shape after filtering: (9020210, 49)


In [59]:
# sorted([col for col in all_data.columns if 'item_name_svd_' not in col])

In [60]:
sorted([col for col in all_data.columns])

['city',
 'city_item_avg_date_sales_lag_1',
 'city_type_avg_date_sales_lag_1',
 'date_block_num',
 'date_item_avg_price_lag_1',
 'decay_rate_feature',
 'delta_price_lag',
 'delta_shop_revenue_lag',
 'item_age',
 'item_avg_date_sales_lag_1',
 'item_avg_date_sales_lag_12',
 'item_avg_date_sales_lag_2',
 'item_avg_date_sales_lag_6',
 'item_avg_price_expanding',
 'item_category_avg_date_sales_lag_1',
 'item_category_expanding_mean',
 'item_category_id',
 'item_cnt_month',
 'item_cnt_month_lag_1',
 'item_cnt_month_lag_12',
 'item_cnt_month_lag_2',
 'item_cnt_month_lag_3',
 'item_cnt_month_lag_6',
 'item_expanding_mean',
 'item_id',
 'item_shop_age',
 'item_shop_last_sale',
 'items_per_transaction_lag_1',
 'meta_type',
 'month',
 'platform_type',
 'rolling_3m_cnt_mean',
 'rolling_3m_cnt_std',
 'rolling_3m_item_mean',
 'rolling_3m_item_std',
 'rolling_3m_price_mean',
 'rolling_3m_price_std',
 'shop_avg_date_sales_lag_1',
 'shop_expanding_mean',
 'shop_id',
 'shop_item_category_avg_date_sales_

## clip
- 데이터에 대한 설명에 따라 상점별 아이템 월간 판매량에 대한 피처들을 (0, 20)으로 클립
- 모델이 극단적인 값에 대해 학습하지 않도록 방지함

In [61]:
cols_to_clip =['item_cnt_month', 'item_cnt_month_lag_1', 'item_cnt_month_lag_12', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3', \
      'item_cnt_month_lag_6', 'rolling_3m_cnt_mean',]

for col in cols_to_clip:
    all_data[col] = all_data[col].clip(0, 20)

# save

In [62]:
all_data.shape

(9020210, 49)

In [63]:
temp = all_data.isna().sum()
temp[temp > 0]

Series([], dtype: int64)

In [64]:
all_data = downcast(all_data)
joblib.dump(all_data, data_path + 'all_data1.joblib')

Memory usage reduced from 1488.21 MB to 1350.57 MB


['./data/all_data1.joblib']

In [65]:
all_data.loc[(all_data['item_cnt_month_lag_1'] == all_data['item_cnt_month'])]['item_cnt_month'].value_counts()

item_cnt_month
0     7577139
1      130668
2       21346
3        6481
4        2669
20       2620
5        1284
6         749
7         471
8         273
9         175
10        125
11         82
12         61
13         45
14         32
15         26
17         17
19         15
18         12
16         10
Name: count, dtype: int64

In [66]:
same_counts = all_data.loc[(all_data['item_cnt_month_lag_1'] == all_data['item_cnt_month'])]['item_cnt_month'].value_counts()
same_counts[0] / len(all_data)

0.8400180261878604

In [67]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9020210 entries, 46 to 15558092
Data columns (total 49 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   date_block_num                           int8   
 1   shop_id                                  int8   
 2   item_id                                  int16  
 3   item_cnt_month                           int8   
 4   city                                     int8   
 5   item_category_id                         int8   
 6   platform_type                            int8   
 7   meta_type                                int8   
 8   type                                     int8   
 9   month                                    int8   
 10  shop_revenue_share_lag_1                 float32
 11  shop_avg_date_sales_lag_1                float32
 12  shop_item_category_avg_date_sales_lag_1  float32
 13  shop_type_avg_date_sales_lag_1           float32
 14  item_category_avg_dat